<a href="https://colab.research.google.com/github/rouyu0405/IAT360/blob/main/Computer_Vision_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sajcabckjakja

NameError: name 'sajcabckjakja' is not defined

In [ ]:
!pip -q install ultralytics==8.3.0 roboflow pandas
from ultralytics import YOLO
from roboflow import Roboflow
import pandas as pd, os, glob


In [ ]:
# 用你自己的 API、workspace、project、version
RF_API_KEY = "YOUR_API_KEY"
RF_WORKSPACE = "your-workspace"          # 例如 "jiawei-shi"
RF_PROJECT   = "fall-pose"               # 你项目的短名 slug
RF_VERSION   = 1                         # 版本号

rf = Roboflow(api_key=RF_API_KEY)
project = rf.workspace(RF_WORKSPACE).project(RF_PROJECT)
dataset = project.version(RF_VERSION).download("yolov8")  # 注意：pose 项目也用 "yolov8" 格式
data_yaml = os.path.join(dataset.location, "data.yaml")
print("data.yaml =", data_yaml)

# 小检查：YOLOv8 Pose 的 data.yaml 通常包含 kpt_shape 等字段
print("\n--- data.yaml 前几行 ---")
print(open(data_yaml).read().splitlines()[:20])


In [ ]:
# 设定一个独立的实验名，便于之后对比
EXP1_NAME = "exp_baseline_b8_lr1e-3"

model = YOLO("yolov8n-pose.pt")  # 关键点检测的预训练权重
results = model.train(
    data=data_yaml,
    epochs=50,          # 课程常用 30~100；先 50
    imgsz=640,
    batch=8,            # baseline：batch=8
    lr0=1e-3,           # baseline：初始学习率
    project="runs_pose",
    name=EXP1_NAME,
    verbose=True
)


In [ ]:
# (A) 验证集评估
val_res = model.val(data=data_yaml, split='val', project="runs_pose", name=f"{EXP1_NAME}_val")
print(val_res)  # 控制台会打印总体指标

# (B) 用测试集出可视化预测图（放进报告）
test_images_dir = os.path.join(dataset.location, "test", "images")
pred_res = model.predict(
    source=test_images_dir,
    conf=0.25,
    save=True,            # 保存可视化
    project="runs_pose",
    name=f"{EXP1_NAME}_pred"
)
print("Predictions saved to:", pred_res[0].save_dir)


In [ ]:
EXP2_NAME = "exp_batch16_lr5e-4"

model2 = YOLO("yolov8n-pose.pt")
results2 = model2.train(
    data=data_yaml,
    epochs=50,
    imgsz=640,
    batch=16,          # 改动1：更大 batch
    lr0=5e-4,          # 改动2：更小学习率（你也可以只改一个超参）
    project="runs_pose",
    name=EXP2_NAME,
    verbose=True
)

# 评估 + 预测（与 baseline 同步生成报告素材）
val_res2 = model2.val(data=data_yaml, split='val', project="runs_pose", name=f"{EXP2_NAME}_val")
test_images_dir = os.path.join(dataset.location, "test", "images")
pred_res2 = model2.predict(
    source=test_images_dir,
    conf=0.25,
    save=True,
    project="runs_pose",
    name=f"{EXP2_NAME}_pred"
)


In [ ]:
def load_last_row(csv_path):
    df = pd.read_csv(csv_path)
    return df.tail(1).assign(csv=os.path.basename(csv_path))

csv1 = f"runs_pose/{EXP1_NAME}/results.csv"
csv2 = f"runs_pose/{EXP2_NAME}/results.csv"

df1 = load_last_row(csv1)
df2 = load_last_row(csv2)
compare = pd.concat([df1, df2], ignore_index=True)

# 常见列名（不同任务略有差异）：precision, recall, map50, map, kpt_mAP50, kpt_mAP50-95 等
print(compare.filter(regex="(precision|recall|f1|map|kpt|csv)", axis=1))


In [ ]:
# 导出 ONNX / TorchScript 等（可选）
best1 = f"runs_pose/{EXP1_NAME}/weights/best.pt"
best2 = f"runs_pose/{EXP2_NAME}/weights/best.pt"
YOLO(best1).export(format="onnx")   # -> runs_pose/exp.../weights/best.onnx
YOLO(best2).export(format="onnx")
